# Preprocessing (for U-Net)
This notebook contains imutils to prepare data to Train Unet.
#Or other things too 

In [11]:
import tifffile as tiff
from natsort import natsorted
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cv2
from imutils.src import imfunctions #contours_length as imutils.contours_length
from natsort import natsorted
import shutil
import glob
import random

In [12]:
from imutils.src.model import *

In [13]:
from centerline.src import make_skeleton

#### Part 1: Extract frames
Part 1 is about extracting any frame. Part II is if you want to extract frames that fullfill a condition.

In [ ]:
#Here I am extracting frames from a specific interval

frames2extract=np.arange(0,200000,17777)#(0,200000,10000)
input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all/'
output_folder='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test/'
try: os.mkdir(output_folder) # creates the subdirectory where it should be stored
except: print('Output Directory already exists, might overwrite')

for btf_file in natsorted(os.listdir(input_folder)):
    print(btf_file)
    if '2020-07' in btf_file:
        print(btf_file)
        btf_file_basename=os.path.splitext(os.path.basename(btf_file))[0]
        print(btf_file_basename)
        imfunctions.extract_frames(os.path.join(input_folder,btf_file), os.path.join(output_folder,btf_file_basename),frames2extract)
        print('end')

In [ ]:
Or using the find_nan_centerlines()

In [18]:
# Or using the find_nan_centerlines()

input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary2/'
output_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/all_wrong_centerlines/'
spline_K_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/skeleton_after_head_and_tail_from_unet_all/'

# the lines below are commented when the directory already exists
# try: os.mkdir(output_folder) # creates the subdirectory where it should be stored
# except: print('Output Directory already exists, might overwrite')

for btf_file in natsorted(os.listdir(input_folder)):
    print(btf_file)
    if '2020-07' in btf_file and btf_file.endswith('.btf'):
        print(btf_file)
        btf_file_basename=os.path.splitext(os.path.basename(btf_file))[0]
        print(btf_file_basename)
        #get name before -channel- string
        file_basename=btf_file_basename.partition('-channel')[0]
        print(file_basename)

        spline_K_root_path=os.path.join(spline_K_folder,file_basename)
        print(spline_K_root_path)
        spline_K_path=glob.glob(spline_K_root_path+'-_spline_K.csv')
        if len(spline_K_path)>1:
            'Print error'
            break
        else: spline_K_path=spline_K_path[0]
        print(spline_K_path)
        wrong_centerlines,correct_centerlines=make_skeleton.find_nan_centerlines(spline_K_path)
        print(len(wrong_centerlines))
        if len(wrong_centerlines)>10:
            random.seed(10)
            subsampled_wrong_centerlines=random.sample(wrong_centerlines,10)
        else: subsampled_wrong_centerlines=wrong_centerlines
        imfunctions.extract_frames(os.path.join(input_folder,btf_file), os.path.join(output_folder,btf_file_basename),frames_list=subsampled_wrong_centerlines)
        #break
print('end')


2020-07-01_10-10-48_control_worm1-channel-0-bigtiff.btf
2020-07-01_10-10-48_control_worm1-channel-0-bigtiff.btf
2020-07-01_10-10-48_control_worm1-channel-0-bigtiff
2020-07-01_10-10-48_control_worm1
/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/skeleton_after_head_and_tail_from_unet_all/2020-07-01_10-10-48_control_worm1
/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/skeleton_after_head_and_tail_from_unet_all/2020-07-01_10-10-48_control_worm1-_spline_K.csv
2876
making  /groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/all_wrong_centerlines/2020-07-01_10-10-48_control_worm1-channel-0-bigtiff  directory
111132
17507
76232
94795
111159
6759
69009
76389
94831
69354
2020-07-01_10-50-18_control_worm2-channel-0-bigtiff.btf
2020-07-01_10-50-18_control_worm2-channel-0-bigtiff.btf
2020-07-01_10-50-18_control_worm2-channel-0-bigtiff
2020-07-01_10-50-18_control_worm2
/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/skeleton_after_head_a

In [46]:
#add zeros (just in case)

path='/groups/zimmer/Ulises//code/unet-master/data/data_preparation/binary/'
len_max_number=6

for dirpath, _, files in os.walk(path):
    print(dirpath)
    imfunctions.add_zeros_to_filename(dirpath, len_max_number)

/groups/zimmer/Ulises//code/unet-master/data/data_preparation/binary/
/groups/zimmer/Ulises//code/unet-master/data/data_preparation/binary/2020-07-01_10-10-48_control_worm1-channel-0-bigtiff
/groups/zimmer/Ulises//code/unet-master/data/data_preparation/binary/2020-07-01_10-50-18_control_worm2-channel-0-bigtiff
/groups/zimmer/Ulises//code/unet-master/data/data_preparation/binary/2020-07-01_11-50-53_control_worm3-channel-0-bigtiff
/groups/zimmer/Ulises//code/unet-master/data/data_preparation/binary/2020-07-01_13-21-00_chemotaxisl_worm1-channel-0-bigtiff
/groups/zimmer/Ulises//code/unet-master/data/data_preparation/binary/2020-07-01_14-41-11_chemotaxisl_worm2-channel-0-bigtiff
/groups/zimmer/Ulises//code/unet-master/data/data_preparation/binary/2020-07-01_15-42-06_chemotaxisl_worm4-channel-0-bigtiff
/groups/zimmer/Ulises//code/unet-master/data/data_preparation/binary/2020-07-01_16-20-36_chemotaxisl_worm5-channel-0-bigtiff
/groups/zimmer/Ulises//code/unet-master/data/data_preparation/binar

In [51]:
#rename to have unique names (prepend folder name into file name)

path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/raw/'
for dirpath, _, files in os.walk(path):
    for f in files:
        os.rename(os.path.join(dirpath, f), os.path.join(dirpath, os.path.split(dirpath)[-1] + '_' + f))

In [52]:
#copy all files from subfolders into a new single folder

#path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_test/'
dst_path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_raw_all/'

try: os.mkdir(dst_path) # creates the subdirectory where it should be stored
except: print('Output Directory already exists, might overwrite')

for directory, _, files in os.walk(path):
    for file in files:
        #print(file)
        shutil.copyfile(os.path.join(directory,file),os.path.join(dst_path,file))


In [ ]:
#substract background if wanted

bg_img_filename='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/background/2020-07-01_13-42-05_chemotaxisl_worm1_bg-channel-0-_MMStack_Median_BG.tiff'
bg_img=tiff.imread(bg_img_filename)

path='/groups/zimmer/Ulises/code/unet-master/data/worm2/test2/'

#this code is flexible to files being in separate folders
for directory, subdirectories, files in os.walk(path):
    for file in files:
        filepath=os.path.join(directory,file)
        img=tiff.imread(filepath)
        inv_img=cv2.bitwise_not(img)
        new_img=cv2.subtract(inv_img,bg_img)
        tiff.imwrite(filepath,new_img)

In [ ]:
#Use draw_some_contours function to extract interesting contours
path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_all_image/'

for directory, subdirectories, files in os.walk(path):
    for file in files:
        filepath=os.path.join(directory,file)
        img=tiff.imread(filepath)
        #median Blur
        img=cv2.medianBlur(img,5)
       
        #apply threshold
        ret, new_img = cv2.threshold(img,20,255,cv2.THRESH_BINARY)
        worm_contour_img=imfunctions.draw_some_contours(new_img,contour_size=8000,tolerance=0.25, inner_contour_area_to_fill=300)
        
        #if you want to plot the different stages
        #plotting
#         fig, (ax1,ax2)=plt.subplots(figsize=(10,5), nrows=1, ncols=2, dpi=100)
#         im=ax1.imshow(img)
#         divider = make_axes_locatable(ax1)
#         cax = divider.append_axes('right', size='5%', pad=0.05)
#         fig.colorbar(im, cax=cax, orientation='vertical')
#         ax2.imshow(worm_contour_img)
#         [axi.set_axis_off() for axi in (ax1,ax2)]
        
        tiff.imwrite(filepath,worm_contour_img)

In [53]:
#convert to stack if wanted to screen for masks that need improvement (eg remove bubble)
path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_raw_all/'
o_path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_raw_all_stack.tif'
imfunctions.images2stack(path, o_path)

In [54]:
#manually annotate frames to correct

#path='/groups/zimmer/Ulises/code/unet-master/data/worm2_background_removed/test/'
idxs=[5,9,10,11,12,14,27,33,34,35,36,37,55,56,57,61,62,63,64,65,66,67,68,69,70,71,72,73,77,78,85,93,94,97]#[53,76,98]#[15,67,80,89,92,113,157,158,169]
print('lenght of indexes to delete is', len(idxs))
#get their filename
for idx,file in enumerate(natsorted(os.listdir(path))):
    #add +1 if idxs are starting from 1
    if idx+1 in idxs:
        print(idx+1, file)

lenght of indexes to delete is 34
5 2020-07-01_10-10-48_control_worm1-channel-0-bigtiff_img076232.tif
9 2020-07-01_10-10-48_control_worm1-channel-0-bigtiff_img111132.tif
10 2020-07-01_10-10-48_control_worm1-channel-0-bigtiff_img111159.tif
11 2020-07-01_10-50-18_control_worm2-channel-0-bigtiff_img013175.tif
12 2020-07-01_10-50-18_control_worm2-channel-0-bigtiff_img013343.tif
14 2020-07-01_10-50-18_control_worm2-channel-0-bigtiff_img126920.tif
27 2020-07-01_11-50-53_control_worm3-channel-0-bigtiff_img078717.tif
33 2020-07-01_13-21-00_chemotaxisl_worm1-channel-0-bigtiff_img076743.tif
34 2020-07-01_13-21-00_chemotaxisl_worm1-channel-0-bigtiff_img076977.tif
35 2020-07-01_13-21-00_chemotaxisl_worm1-channel-0-bigtiff_img077011.tif
36 2020-07-01_13-21-00_chemotaxisl_worm1-channel-0-bigtiff_img077032.tif
37 2020-07-01_13-21-00_chemotaxisl_worm1-channel-0-bigtiff_img077083.tif
55 2020-07-01_16-20-36_chemotaxisl_worm5-channel-0-bigtiff_img079325.tif
56 2020-07-01_16-20-36_chemotaxisl_worm5-channe

In [58]:
#delete the files of the corresponding indexes
path='/groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_raw_all_corrected/'
idxs=[5,9,10,11,12,14,27,33,34,35,36,37,55,56,57,61,62,63,64,65,66,67,68,69,70,71,72,73,77,78,85,93,94,97]
for idx,file in enumerate(natsorted(os.listdir(path))):
    #add +1 if idxs are starting from 1
    if idx+1 in idxs:
        filepath=os.path.join(path,file)
        print('you will delete', idx+1, filepath)

        os.remove(filepath)

you will delete 5 /groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_raw_all_corrected/2020-07-01_10-10-48_control_worm1-channel-0-bigtiff_img076232.tif
you will delete 9 /groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_raw_all_corrected/2020-07-01_10-10-48_control_worm1-channel-0-bigtiff_img111132.tif
you will delete 10 /groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_raw_all_corrected/2020-07-01_10-10-48_control_worm1-channel-0-bigtiff_img111159.tif
you will delete 11 /groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_raw_all_corrected/2020-07-01_10-50-18_control_worm2-channel-0-bigtiff_img013175.tif
you will delete 12 /groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_raw_all_corrected/2020-07-01_10-50-18_control_worm2-channel-0-bigtiff_img013343.tif
you will delete 14 /groups/zimmer/Ulises/code/unet-master/data/data_preparation/new_training_raw_all_corrected/2020-0

#### Part 2
Extract only frames where there is a coiled shape (contour have children)

**Note**: There is also the function find_nan_centerlines() in the centerline package, which outputs the selected frames that then can be extracted with extract_frames()

In [ ]:
#for very file in binary_all_folder:
main_input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary2_eroded_twice/'
output_folder='/groups/zimmer/Ulises/code/unet-master/data/a_new_crop_worm_eroded_twice_linewidth/train/label/'

for input_filepath in os.listdir(main_input_folder):
    #if input_filepath.endswith('.*tif*'):# and '2020' in input_filepath:
    if '2020-07' in input_filepath:
        file_basename=os.path.splitext(os.path.basename(input_filepath))[0]
        with tiff.TiffFile(os.path.join(main_input_folder,input_filepath)) as tif:
            for i, page in enumerate(tif.pages):#enumerate(tif.series):#enumerate(tif.series[0].pages):
                if i%250!=0: continue
                img=page.asarray()
        #         plt.imshow(img)
        #         plt.show()
                contours_with_children=imfunctions.extract_contours_with_children(img)

                for cnt_idx,cnt in enumerate(contours_with_children):
                        x,y,w,h = cv2.boundingRect(cnt)
                        #make the crop
                        cnt_img=img[y:y+h,x:x+w]
        #                 plt.imshow(cnt_img)
        #                 plt.show()
                        tiff.imwrite(os.path.join(output_folder,file_basename+'_frame_'+str(i)+'_cnt_'+str(cnt_idx)+'.tiff'),cnt_img)
                #if i>20000: break
        

In [ ]:
#TO get bounding boxs of an image that is not binary but has a binarized sister file

raw_input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary2_eroded_twice/'
binary_input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary2'#/'

output_folder='/groups/zimmer/Ulises/code/unet-master/data/a_new_crop_worm_eroded_twice_linewidth/train/label_bu/'

for input_filepath in os.listdir(raw_input_folder):
    #if input_filepath.endswith('.*tif*'):# and '2020' in input_filepath:
    if '2020-07' in input_filepath:
        file_basename=os.path.splitext(os.path.basename(input_filepath))[0]
        input_raw=os.path.join(raw_input_folder,input_filepath)
        input_binary=os.path.join(binary_input_folder,input_filepath)
        output_filepath=os.path.join(output_folder,input_filepath)
        #declare readers
        with tiff.TiffFile(input_binary) as tif_binary, tiff.TiffFile(input_raw) as tif_raw:
            for i, page in enumerate(tif_binary.pages):#enumerate(tif.series):#enumerate(tif.series[0].pages):
                if i%250!=0: continue
                img=page.asarray()

                contours_with_children=imfunctions.extract_contours_with_children(img)
                
                for cnt_idx,cnt in enumerate(contours_with_children):
                    x,y,w,h = cv2.boundingRect(cnt)
                    #make the crop
                    raw_img=tif_raw.pages[i].asarray()
                    cnt_img=raw_img[y:y+h,x:x+w]

                    tiff.imwrite(os.path.join(output_folder,file_basename+'_frame_'+str(i)+'_cnt_'+str(cnt_idx)+'.tiff'),cnt_img)
            

In [ ]:
img_path='/groups/zimmer/Ulises/code/unet-master/data/crop_worm_raw/train/label/2020-07-01_17-32-12_control_worm4-channel-0-bigtiff_frame_60000_raw_cnt_0.tiff'
img=tiff.imread(img_path)
plt.imshow(img)

## Compare image vs label

In [ ]:

train_folder='/groups/zimmer/Ulises/code/unet-master/data/a_new_crop_worm_eroded/train'
image_path=os.path.join(train_folder,'image')
label_path=os.path.join(train_folder,'label')
label_mask_path=os.path.join(train_folder,'label_mask')


for i,file in enumerate(natsorted(os.listdir(image_path))):
    #print(file)
    #if i<225:continue
    fig, axes=plt.subplots(ncols=3,dpi=150)
    image=tiff.imread(os.path.join(image_path, file))
    label=tiff.imread(os.path.join(label_path, file))
    label_mask=tiff.imread(os.path.join(label_mask_path, file))

    axes[0].imshow(image)
    axes[1].imshow(label_mask)
    axes[2].imshow(label)
    for ax in axes:
        ax.set_axis_off()
    #fig.suptitle(str(i)+file)
    

### Part 3
Cut everywhere where there are contours with children, run the network and paste them back

In [ ]:
import tifffile as tiff
from natsort import natsorted
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cv2
from imutils.src import imfunctions #contours_length as imutils.contours_length
from natsort import natsorted
import shutil

from imutils.src.model import *

In [ ]:
#for very file in binary_all_folder:
main_input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary/'
output_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary_after_unet/'

weights_path='/groups/zimmer/Ulises/code/unet-master/data/crop_worm/training_results/24051180_w_validation500steps_10epochs/unet_master.hdf5'

#unet has to have been imported
model=unet()
model.load_weights(weights_path)

for input_filepath in os.listdir(main_input_folder):
    #if input_filepath.endswith('.*tif*'):# and '2020' in input_filepath:
    if '2020-07' in input_filepath:
        file_basename=os.path.splitext(os.path.basename(input_filepath))[0]
        with tiff.TiffFile(os.path.join(main_input_folder,input_filepath)) as tif:
            for i, page in enumerate(tif.pages):#enumerate(tif.series):#enumerate(tif.series[0].pages):
                if i>30000:break
                if i%50!=0: continue
                img=page.asarray()
        #         plt.imshow(img)
        #         plt.show()
                contours_with_children=imfunctions.extract_contours_with_children(img)

                for cnt_idx,cnt in enumerate(contours_with_children):
                        x,y,w,h = cv2.boundingRect(cnt)
                        #make the crop
                        cnt_img=img[y:y+h,x:x+w]
                        
                        #run U-Net network:
                        if cnt_img.shape!=(256,256):
                            cnt_img=cv2.resize(cnt_img, (256,256))
                            print('original image has been resized to: ', cnt_img.shape)
                        cnt_img=np.reshape(cnt_img,cnt_img.shape+(1,))
                        cnt_img=np.reshape(cnt_img,(1,)+cnt_img.shape)

                        print('New image shape is: ', cnt_img.shape)
                        cnt_img=cnt_img/255
                        results = model.predict(cnt_img)
                        print('Results image shape is: ', results.shape)
                        #reshape results
                        results_reshaped=results.reshape(256,256)
                        #resize results
                        results_reshaped=cv2.resize(results_reshaped, (w,h))
                        #multiply it by 255
                        results_reshaped=results_reshaped*255
                        
                        #paste it into the original image
                        new_img=img.copy()
                        new_img[y:y+h,x:x+w]=results_reshaped
                        
                        #visualize
#                         fig,(ax0,ax1,ax2,ax3,ax4,ax5)=plt.subplots(figsize=(18,3), ncols=5)
#                         ax0.imshow(img)
#                         ax1.imshow(img[y:y+h,x:x+w])
#                         ax2.imshow(cnt_img.reshape((256,256)))
#                         ax3.imshow(results.reshape((256,256)))
#                         ax4.imshow(results_reshaped)
#                         ax5.imshow(img)
#                         plt.show()
#                         fig,(ax0,ax1)=plt.subplots(figsize=(6,3), ncols=2,dpi=150)
#                         ax0.imshow(img)
#                         ax1.imshow(new_img, vmin=240)
#                         plt.show()
                        
#                         break
        #                 plt.imshow(cnt_img)
        #                 plt.show()
                        tiff.imwrite(os.path.join(output_folder,file_basename+'_frame_'+str(i)+'_cnt_'+str(cnt_idx)+'.tiff'),cnt_img)
                #if i>20000: break

In [ ]:
binary_input_filepath='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary2_eroded/2020-07-01_18-36-25_control_worm6-channel-0-bigtiff.btf'
raw_input_filepath='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_background_substracted/2020-07-01_18-36-25_control_worm6-channel-0-bigtiff.btf'
output_filepath='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/ot_test2.tif'
weights_path='/groups/zimmer/Ulises/code/unet-master/data/a_new_crop_worm_eroded/training_results/24264146_w_validation500steps_50epochs/unet_master.hdf5'

unet_segmentation_contours_with_children(binary_input_filepath, raw_input_filepath, output_filepath, weights_path)


In [ ]:
#this function is already in imutils
def unet_segmentation_contours_with_children(binary_input_filepath, raw_input_filepath, output_filepath, weights_path):
    """
    Run through the unet segmentation the contours with children.
    TO DO: It would be more efficient to do a list of frames.
    Parameters:
    -----------
    input_filepath, str
    output_filepath, str
    weights_path, str
    Returns:
    -----------

    """

    model=unet()
    model.load_weights(weights_path)

    with tiff.TiffFile(binary_input_filepath, multifile=False) as binary_tif,\
     tiff.TiffFile(raw_input_filepath, multifile=False) as raw_tif,\
     tiff.TiffWriter(output_filepath, bigtiff=True) as tif_writer:
        for i, page in enumerate(binary_tif.pages):
            #if i% 5!=0: continue
            #if i<400: continue
            if i>405: break
            img=page.asarray()
            
            #find contours
            _,cnts,hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            plt.figure(dpi=100)
            plt.imshow(img)
            plt.title(i)
            #plt.show()
            
            #if there is None or less than 2 contours: continue
            if cnts is None or len(cnts)<2:
                tif_writer.write(img, contiguous=True)
                continue

            #find contours with children
            contours_with_children=extract_contours_with_children(img)

            #make a copy of the original image here in order to paste more than one contour with children
            new_img=raw_tif.pages[i].asarray()
            for cnt_idx,cnt in enumerate(contours_with_children):
                    x,y,w,h = cv2.boundingRect(cnt)
                    #make the crop
                    cnt_img=new_img[y:y+h,x:x+w]

                    #run U-Net network:
                    cnt_img=cv2.resize(cnt_img, (256,256))
                    cnt_img=np.reshape(cnt_img,cnt_img.shape+(1,))
                    cnt_img=np.reshape(cnt_img,(1,)+cnt_img.shape)

                    #normalize to 1 by dividing by 255
                    cnt_img=cnt_img/255
                    results = model.predict(cnt_img)
                    #reshape results
                    results_reshaped=results.reshape(256,256)
                    #resize results
                    results_reshaped=cv2.resize(results_reshaped, (w,h))
                    #multiply it by 255
                    results_reshaped=results_reshaped*255

                    #paste it into the original image
                    new_img[y:y+h,x:x+w]=results_reshaped
#                     plt.imshow(new_img)
#                     plt.show()
#                     plt.title(i)

            tif_writer.write(new_img, contiguous=True)


### Trying eroding mask

In [ ]:
from skimage.morphology import binary_erosion
import tifffile as tiff

In [ ]:
#this exists as a function in imutils
path='/groups/zimmer/Ulises/code/unet-master/data/crop_worm_eroded/train/label/'
eroded_path='/groups/zimmer/Ulises/code/unet-master/data/crop_worm_eroded/train/label_eroded2/'

for file in (os.listdir(path)):
    img=tiff.imread(os.path.join(path,file))
    eroded=binary_erosion(img)
    eroded=binary_erosion(eroded)
    #conver to image with values form 0 to 255
    eroded = eroded.astype(np.uint8)  #convert to an unsigned byte
    eroded*=255
    fig, (ax0,ax1)=plt.subplots(ncols=2)
    ax0.imshow(img)
    ax1.imshow(eroded)
    tiff.imwrite(os.path.join(eroded_path,file), eroded)
    

In [ ]:
#for the btf:
input_filepath='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary2/2020-07-01_18-36-25_control_worm6-channel-0-bigtiff.btf'
output_filepath='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary_eroded/2020-07-01_18-36-25_control_worm6-channel-0-bigtiff_5.btf'
erosion_rounds=1
erode(input_filepath,output_filepath,1)

In [ ]:
#add zeros (just in case)
import os
path='/groups/zimmer/Ulises/code/unet-master/data/a_new_crop_worm_eroded_twice_linewidth/train/label/'
for dirpath, _, files in os.walk(path):
    print(dirpath)
    new_add_zeros_to_filename(dirpath, 6)

In [ ]:
def new_add_zeros_to_filename(path, len_max_number=6):
    """
    Change the filename of images from img235.png to img00235.png depending on len_max_number
    It has a sister function: add_zeros_to_csv
    Parameters:
    -----------
    path: str,
        Path to the directory with the images
    len_max_number: int
        number of digits the number should have, default is 6
    """
    #creates numberic regular expression
    regex_num=re.compile(r'\d+')

    files=os.listdir(path)
    #this could be improved with a regular expression to catch the numbers.
    for filename in files:
        #if 'img' not in filename: continue
        #print(filename)
        new_filename=filename
        
        file_start, file_end = filename.split('frame_')
        
        number=regex_num.search(file_end).group(0)

        #get the file exntesion without the do (e.g. 'png') 
        file_extension=re.split('\.',filename)[-1]
        
        #while the number is smaller than the len_max_number
        zeros=''
        while len(zeros+number) < len_max_number:
            #print(re.split('img', filename)[1])
            zeros=zeros+'0'
        #new filename= img string+number+ dot + extension
        new_filename=file_start+'frame_'+zeros+file_end
        os.rename(os.path.join(path, filename), os.path.join(path, new_filename))

### Segment with Napari

In [ ]:
#has to be locally

In [ ]:
viewer = napari.view_image(data)
viewer.add_labels(segmentation)

### Binarize your U-net output

In [ ]:
input_path='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/skeleton_after_new_unet/2020-07-01_18-36-25_control_worm6-channel-0-bigtiff.btf'
output_path='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/skeleton_after_new_unet/2020-07-01_18-36-25_control_worm6-channel-0-bigtiff_binary_200th.btf'
with tiff.TiffFile(input_path) as tif, tiff.TiffWriter(output_path, bigtiff=True) as tif_writer:
    for page in tif.pages:
        img=page.asarray()
        ret, new_img = cv2.threshold(img,200,255,cv2.THRESH_BINARY)
        tif_writer.write(new_img,contiguous=True)

In [ ]:
## Charlie s idea

In [ ]:
train_folder='/groups/zimmer/Ulises/code/unet-master/data/a_new_crop_worm_eroded_linewidth3_negative/train/'
label_path=os.path.join(train_folder,'label')
label_mask_path=os.path.join(train_folder,'label_mask')

output=os.path.join(train_folder,'new_label')
for i,file in enumerate(natsorted(os.listdir(image_path))):
    print(file)
    fig, axes=plt.subplots(ncols=3)
    label=tiff.imread(os.path.join(label_path, file))
    label_mask=tiff.imread(os.path.join(label_mask_path, file))
    
    new_img=label_mask-label
    axes[0].imshow(label_mask)
    axes[1].imshow(label)
    axes[2].imshow(new_img)
    plt.show()
    #if i>10: break
    tiff.imwrite(os.path.join(output,file),new_img)

In [ ]:
new_img

In [ ]:
path='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary_after_unet_raw_24264146_w_validation500steps_50epochs/2020-07-01_18-36-25_control_worm6-channel-0-bigtiff.btf'
with tiff.TiffFile(path) as tif:
    for i, page in enumerate(tif.pages):
        if i<400: continue
        img=page.asarray()
        plt.imshow(img)
        plt.show()
        plt.title(i)
        if i>450: break

In [ ]:
path='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary2_eroded_binarized/2020-07-01_18-36-25_control_worm6-channel-0-bigtiff.btf'
with tiff.TiffFile(path) as tif:#, tiff.TiffFile(path_raw) as tif_raw:
    for i, page in enumerate(tif.pages):
        if i<400: continue
        img=page.asarray()
        _,cnts,hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        if len(cnts)>1:
            print('hello', i)
            new_img=np.zeros((610,610))

            for i in range(len(cnts)):
                plt.figure(dpi=100)
                cv2.drawContours(new_img,cnts,i,(255,255,255),5)
                plt.imshow(new_img)
                plt.show()
        plt.figure(dpi=100)
        plt.imshow(img)
        #plt.show()
        plt.title(i)
        if i>405: break

## Extract Frames and save them into on tiff stack
This code was in extract_frames.ipynb before.

In [ ]:
#this is a coiled shape dict for worm:
#2020-07-01_18-36-25_control_worm6-channel-
coiled_shapes_dict={'ot1.tif':np.arange(2800,5603),
                    'ot2.tif':np.arange(6676,7300),
                    'ot3.tif':np.arange(18500,21000),
                    'ot4.tif':np.arange(21600,22345),
                    'ot5.tif':np.arange(24300,24900),
                    'ot6.tif':np.arange(27616,28200),
                    'ot7.tif':np.arange(34588,35400),
                    'ot8.tif':np.arange(38980,40000),
                    'ot9.tif':np.arange(47800,48600),
                    'ot10.tif':np.arange(59300,60100),
                    'ot11.tif':np.arange(62150,63340),
                    'ot12.tif':np.arange(93780,94100),
                    'ot13.tif':np.arange(108150,109350),
                    'ot14.tif':np.arange(116500,117200),
                    'ot15.tif':np.arange(163100,163600),
                    'ot16.tif':np.arange(165660,166690),
                    'ot17.tif':np.arange(171200,172400),
                    'ot18.tif':np.arange(172900,173960),
                    'ot19.tif':np.arange(194195,195000)}

In [ ]:
#main_path='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/skeleton_before_unet/'#btf_all_binary_after_unet_binary_24176635_w_validation500steps_40epochs/'
#input_filepath='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all_binary_after_unet_binary_24176635_w_validation500steps_40epochs/2020-07-01_18-36-25_control_worm6-channel-0-bigtiff.btf'
path='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/skeleton_after_new_unet/'

for input_filepath,_,files in os.walk(path):
    #if 'labeled' not in input_filepath: continue
    for file in files:
        if 'control_worm6' not in file: continue
        if file.endswith('labeled.btf'):
            file_path=os.path.join(input_filepath,file)
            print(file_path)
            with tiff.TiffFile(file_path, multifile=False) as tif:
                for keys, frames in coiled_shapes_dict.items():
                    print(keys)
                    with tiff.TiffWriter(os.path.join(input_filepath,keys),bigtiff=True) as tiff_writer:
                        for frame in frames:
                            img=tif.pages[frame].asarray()
                            tiff_writer.write(img, contiguous=True)

In [ ]:
#for single file (not ready yet)
path='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/skeleton_after_new_unet/'

with tiff.TiffFile(file, multifile=False) as tif:
    for keys, frames in coiled_shapes_dict.items():
        with tiff.TiffWriter(os.path.join(input_filepath,keys),bigtiff=True) as tiff_writer:
            for frame in frames:
                img=tif.pages[frame].asarray()
                tiff_writer.write(img, photometric='minisblack', contiguous=True)

## Extras

Binarize training label

(This does not work because it creates edge artifacts)

In [ ]:
from skimage.morphology import binary_erosion

path='/groups/zimmer/Ulises/code/unet-master/data/a_new_crop_worm_erodedtwice_linewidth/label_binary_mask/'
output_path='/groups/zimmer/Ulises/code/unet-master/data/a_new_crop_worm_erodedtwice_linewidth/label_binary_mask_eroded/'
for dirpath, _, files in os.walk(path):
    for f in files: 

        img=tiff.imread(os.path.join(path,f))
        eroded_img=binary_erosion(img)

        output_filepath=os.path.join(output_path,f)
        tiff.imwrite(output_filepath, eroded_img)